# Bluetooth with Bluetoothctl
This wrapper is inspired by wrapper I saw based on ReachView from Egor Fedorov (egor.fedorov@emlid.com).
I focused this version on functionality analysing the outuput of any commands: for each command sends, the output that came out is parsed to understand what happened.
This work is inspired by 
- https://gist.github.com/egorf/66d88056a9d703928f93
- https://gist.github.com/dmeulen/02591532170ce3b5734946452cea38a1
- https://forums.raspberrypi.com/viewtopic.php?t=170680

## Environment

In [5]:
!python --version

Python 3.12.3


In [6]:
!bluetoothctl --version

bluetoothctl: 5.72


In [10]:
!jupyter lab --version

4.2.5


In [11]:
!jupyter notebook --version

7.2.2


Installing rich package for embellish my log

In [1]:
!pip install rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.6/241.6 kB 1.2 MB/s eta 0:00:001.2 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.1 MB/s eta 0:00:00


## My making off
Remember that I don't want to care about frivolities such as unicode interpretation of colours in bluettothctl console. I just want my code is efficient and manageing buetoothctl in what I need and most common feature used.
Here the base class I use to understand how to work. 

In [62]:
# Based on ReachView code from Egor Fedorov (egor.fedorov@emlid.com)
# Developed and tested with Python 3.12.3 and brluetoothctl versin 5.72
import time
import pexpect
import subprocess
import sys
import re
import logging
# Needed to see log in notebook
from logging import StreamHandler
from rich.logging import RichHandler

logging.basicConfig(level=logging.DEBUG, handlers=[RichHandler()],format='%(asctime)s - %(levelname)s - %(message)s')
handler = StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger = logging.getLogger("Bluetoothctl")
logger.addHandler(handler)

class Bluetoothctl:
    """A wrapper for bluetoothctl utility."""

    def __init__(self):
        self.output=[]
        subprocess.check_output("rfkill unblock bluetooth", shell=True)
        self.process = pexpect.spawnu("bluetoothctl", echo=False)
        logging.debug("Process created")
        #self.output.append(self.process.read())
        
            
    def send(self, command, expected=[], pause=2):
        self.process.send(f"{command}\n")
        time.sleep(pause)
        # Default expected in output
        if "#" not in expected:
            expected.append("#")
        if pexpect.EOF not in expected:
            expected.append(pexpect.EOF)
        logging.info(f"Sending command [{command}] with expected=[{expected}] and pause=[{pause}]")
        if self.process.expect(expected):
            logging.debug(f"Failed after {command}")
            raise Exception(f"failed after {command}")
        else:
            out=self.process.before
            logging.debug(f"Adding to output:[{out}]")
            self.output.append(self.process.before)
            
    def get_output(self, *args, **kwargs):
        """Run a command in bluetoothctl prompt, return output as a list of lines."""
        try:
            self.send(*args, **kwargs)
        except Exception as e:
            raise e   
        return self.read_last_output()

    def read_last_output(self):
        return self.output[-1]

    def read_full_output(self):
        return self.output

Here a function to see output as raw unicode string 

In [13]:
def printunicode(str):
    print("---RAW-UNICODE-RESULT-----------------------------------")
    print(res.encode("raw_unicode_escape"))

Going on initializing the wrapper

In [63]:
btctl=Bluetoothctl()

[09/29/24 17:01:46] DEBUG    2024-09-29 17:01:46,437 - DEBUG - Process created                     ]8;id=239426;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=931576;file:///tmp/ipykernel_16595/4091806857.py#27\27]8;;\

Let's try with help command, first you'll see how debug print output...

In [22]:
# get help
res=btctl.get_output("help",pause=3)

[09/29/24 15:58:18] INFO     2024-09-29 15:58:18,457 - INFO - Sending command [help] with           ]8;id=108312;file:///tmp/ipykernel_16595/508631921.py\508631921.py]8;;\:]8;id=68282;file:///tmp/ipykernel_16595/508631921.py#37\37]8;;\
                             expected=[['#', <class 'pexpect.exceptions.EOF'>, '#', <class                         
                             'pexpect.exceptions.EOF'>, '#', <class 'pexpect.exceptions.EOF'>, '#',                
                             <class 'pexpect.exceptions.EOF'>, '#', <class                                         
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

                    DEBUG    2024-09-29 15:58:18,463 - DEBUG - Adding to output:[ [?2004l           ]8;id=331762;file:///tmp/ipykernel_16595/508631921.py\508631921.py]8;;\:]8;id=322994;file:///tmp/ipykernel_16595/508631921.py#43\43]8;;\
                             [1;39mMenu main:[0m                                                                   
                             [1;39mAvailable commands:[0m                                                          
                             [1;39m-------------------[0m                                                          
                             [0;94madvertise                                         [0mAdvertise                  
                             Options Submenu                                                                       
                             [0;94mmonitor                                                                         
                             [0mAdvertisement Monitor Options Submenu                                              
                             [0;94mscan                                              [0mScan                       
                             Options Submenu                                                                       
                             [0;94mgatt                                              [0mGeneric                    
                             Attribute Submenu                                                                     
                             [0;94madmin                                             [0mAdmin                      
                             Policy Submenu                                                                        
                             [0;94mplayer                                            [0mMedia                      
                             Player Submenu                                                                        
                             [0;94mendpoint                                          [0mMedia                      
                             Endpoint Submenu                                                                      
                             [0;94mtransport                                         [0mMedia                      
                             Transport Submenu                                                                     
                             [0;94mmgmt                                              [0mManagement                 
                             Submenu                                                                               
                             [0;94mmonitor                                                                         
                             [0mAdvertisement Monitor Submenu                                                      
                             [1;39mlist                                              [0mList                       
                             available controllers                                                                 
                             [1;39mshow [ctrl]                                       [0mController                 
                             information                                                                           
                             [1;39mselect <ctrl>                                     [0mSelect                     
                             default controller                                                                    
                             [1;39mdevices [Paired/Bonded/Trusted/Connected]         [0mList                       
                             available devices, with an optional property as the filter                            
                             [1;39msystem-alias <name>                               [0mSet                        
      

Now how output is returned by method as interpretated unicode

In [23]:
print(res)

 
Menu main:
Available commands:
-------------------
advertise                                         Advertise Options Submenu
monitor                                           Advertisement Monitor Options Submenu
scan                                              Scan Options Submenu
gatt                                              Generic Attribute Submenu
admin                                             Admin Policy Submenu
player                                            Media Player Submenu
endpoint                                          Media Endpoint Submenu
transport                                         Media Transport Submenu
mgmt                                              Management Submenu
monitor                                           Advertisement Monitor Submenu
list                                              List available controllers
show [ctrl]                                       Controller information
select <ctrl>                                   

An then as it appears in unicode

In [24]:
printunicode(res)

---RAW-UNICODE-RESULT-----------------------------------
b' \x1b[?2004l\r\r\n\x1bMenu main:\x1b\r\n\x1bAvailable commands:\x1b\r\n\x1b-------------------\x1b\r\n\x1badvertise                                         \x1bAdvertise Options Submenu\r\n\x1bmonitor                                           \x1bAdvertisement Monitor Options Submenu\r\n\x1bscan                                              \x1bScan Options Submenu\r\n\x1bgatt                                              \x1bGeneric Attribute Submenu\r\n\x1badmin                                             \x1bAdmin Policy Submenu\r\n\x1bplayer                                            \x1bMedia Player Submenu\r\n\x1bendpoint                                          \x1bMedia Endpoint Submenu\r\n\x1btransport                                         \x1bMedia Transport Submenu\r\n\x1bmgmt                                              \x1bManagement Submenu\r\n\x1bmonitor                                           \x1bAdvertisement

Going on with list of controller in each format

In [64]:
# get list of controllers
#btctl=Bluetoothctl()
res=btctl.get_output("list")
print(res)
printunicode(res)

[09/29/24 17:01:57] INFO     2024-09-29 17:01:57,969 - INFO - Sending command [list] with          ]8;id=830125;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=72596;file:///tmp/ipykernel_16595/4091806857.py#39\39]8;;\
                             expected=[['#', <class 'pexpect.exceptions.EOF'>]] and pause=[2]                      

                    DEBUG    2024-09-29 17:01:57,976 - DEBUG - Adding to output:[[?2004hAgent      ]8;id=839689;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=82103;file:///tmp/ipykernel_16595/4091806857.py#45\45]8;;\
                             registered                                                                            
                             [?2004l                                                                               
                             Controller 4C:49:6C:0C:6B:4B linuxlite [default]                                      
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

Agent registered

Controller 4C:49:6C:0C:6B:4B linuxlite [default]
[ZX-K22 BT5.1]
---RAW-UNICODE-RESULT-----------------------------------
b'\x1b[?2004hAgent registered\r\n\x1b[?2004l\r\r\nController 4C:49:6C:0C:6B:4B linuxlite [default]\r\n\x1b[?2004h\x1b[ZX-K22 BT5.1]\x1b'


Let's see wich devices are connected

In [65]:
# get device connected
#btctl = Bluetoothctl()
res=btctl.get_output("devices Connected")
print(res)

[09/29/24 17:02:05] INFO     2024-09-29 17:02:05,331 - INFO - Sending command [devices Connected]  ]8;id=942674;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=224852;file:///tmp/ipykernel_16595/4091806857.py#39\39]8;;\
                             with expected=[['#', <class 'pexpect.exceptions.EOF'>]] and pause=[2]                 

                    DEBUG    2024-09-29 17:02:05,339 - DEBUG - Adding to output:[ [?2004l          ]8;id=668174;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=882196;file:///tmp/ipykernel_16595/4091806857.py#45\45]8;;\
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[ZX-K22 BT5.1]


Now trusted devices

In [66]:
# get device truested
#btctl = Bluetoothctl()
res=btctl.get_output("devices Trusted")
print(res)

[09/29/24 17:02:13] INFO     2024-09-29 17:02:13,184 - INFO - Sending command [devices Trusted]    ]8;id=31108;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=931710;file:///tmp/ipykernel_16595/4091806857.py#39\39]8;;\
                             with expected=[['#', <class 'pexpect.exceptions.EOF'>]] and pause=[2]                 

                    DEBUG    2024-09-29 17:02:13,191 - DEBUG - Adding to output:[ [?2004l          ]8;id=560211;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=937265;file:///tmp/ipykernel_16595/4091806857.py#45\45]8;;\
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[ZX-K22 BT5.1]


And now paired devices

In [67]:
# get device paired
#btctl = Bluetoothctl()
res=btctl.get_output("devices Paired")
print(res)

[09/29/24 17:02:42] INFO     2024-09-29 17:02:42,704 - INFO - Sending command [devices Paired]     ]8;id=841304;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=660008;file:///tmp/ipykernel_16595/4091806857.py#39\39]8;;\
                             with expected=[['#', <class 'pexpect.exceptions.EOF'>]] and pause=[2]                 

                    DEBUG    2024-09-29 17:02:42,709 - DEBUG - Adding to output:[ [?2004l          ]8;id=504893;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=765370;file:///tmp/ipykernel_16595/4091806857.py#45\45]8;;\
                             Device E5:1C:60:76:25:4B BT5.1 Mouse                                                  
                             Device 98:D3:31:00:0A:F6 Philips TAH4205                                              
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
Device E5:1C:60:76:25:4B BT5.1 Mouse
Device 98:D3:31:00:0A:F6 Philips TAH4205
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[ZX-K22 BT5.1]


But what if we passed an unknown command?

In [36]:
#Trying with unkown commands
# get device paired
#btctl = Bluetoothctl()
res=btctl.get_output("ciao")
print(res)
res=btctl.get_output("ciao ciao")
print(res)
res=btctl.get_output("devices ciao")
print(res)


[09/29/24 16:14:34] INFO     2024-09-29 16:14:34,304 - INFO - Sending command [ciao] with          ]8;id=57869;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=101890;file:///tmp/ipykernel_16595/4091806857.py#39\39]8;;\
                             expected=[['#', <class 'pexpect.exceptions.EOF'>]] and pause=[2]                      

                    DEBUG    2024-09-29 16:14:34,309 - DEBUG - Adding to output:[ [?2004l          ]8;id=214855;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=889940;file:///tmp/ipykernel_16595/4091806857.py#45\45]8;;\
                             [1;39mInvalid command in menu main: ciao[0m                                           
                             [1;39m                                                                                
                             Use "help" for a list of available commands in a menu.                                
                             Use "menu <submenu>" if you want to enter any submenu.                                
                             Use "back" if you want to return to menu main.[0m                                     
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
Invalid command in menu main: ciao

Use "help" for a list of available commands in a menu.
Use "menu <submenu>" if you want to enter any submenu.
Use "back" if you want to return to menu main.
[ZX-K22 BT5.1]


[09/29/24 16:14:36] INFO     2024-09-29 16:14:36,365 - INFO - Sending command [ciao ciao] with     ]8;id=793964;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=178951;file:///tmp/ipykernel_16595/4091806857.py#39\39]8;;\
                             expected=[['#', <class 'pexpect.exceptions.EOF'>]] and pause=[2]                      

                    DEBUG    2024-09-29 16:14:36,370 - DEBUG - Adding to output:[ [?2004l          ]8;id=186402;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=381379;file:///tmp/ipykernel_16595/4091806857.py#45\45]8;;\
                             [1;39mInvalid command in menu main: ciao[0m                                           
                             [1;39m                                                                                
                             Use "help" for a list of available commands in a menu.                                
                             Use "menu <submenu>" if you want to enter any submenu.                                
                             Use "back" if you want to return to menu main.[0m                                     
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
Invalid command in menu main: ciao

Use "help" for a list of available commands in a menu.
Use "menu <submenu>" if you want to enter any submenu.
Use "back" if you want to return to menu main.
[ZX-K22 BT5.1]


[09/29/24 16:14:38] INFO     2024-09-29 16:14:38,423 - INFO - Sending command [devices ciao] with  ]8;id=482426;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=53901;file:///tmp/ipykernel_16595/4091806857.py#39\39]8;;\
                             expected=[['#', <class 'pexpect.exceptions.EOF'>]] and pause=[2]                      

                    DEBUG    2024-09-29 16:14:38,429 - DEBUG - Adding to output:[ [?2004l          ]8;id=752261;file:///tmp/ipykernel_16595/4091806857.py\4091806857.py]8;;\:]8;id=469638;file:///tmp/ipykernel_16595/4091806857.py#45\45]8;;\
                             Invalid argument ciao                                                                 
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
Invalid argument ciao
[ZX-K22 BT5.1]


We note that if main command is unknown the outut is "Invalid command ..." and if argument is unkown output is "Invalid argument ...".
We have to change code in our class to manage this situations. For example we add thes to command specific expected string and remember wich one matched output

## Updating the core functions of my class

In [101]:
class Bluetoothctl:
    """A wrapper for bluetoothctl utility."""
    expected_common=["Invalid argument","Invalid command","#",pexpect.EOF]
    expected_matched=None
    def __init__(self):
        self.output=[]
        subprocess.check_output("rfkill unblock bluetooth", shell=True)
        self.process = pexpect.spawnu("bluetoothctl", echo=False)
        logger.debug("Process created")
        expected=["Agent registered"]+self.expected_common
        expected_index=self.process.expect(expected)
         
    def send(self, command, expected=[], pause=3):
        """ This method is the core of the class. It sends command to the bluetoothctl process and undetand when an answer arrived """
        self.expected_matched=None
        self.process.send(f"{command}\n")
        time.sleep(pause)
        # Default expected in output, I don't want repetetion
        for common_ex in self.expected_common:
            if common_ex not in expected:
                expected.append(common_ex)
        logging.info(f"Sending command [{command}] with expected=[{expected}] and pause=[{pause}]")
        expected_index=self.process.expect(expected)
        logging.debug(f"expected_index={expected_index}")
        if expected_index<0:
            logging.warn(f"Failed after {command}",e)
            raise Exception(f"failed after {command}")
        else:
            out=self.process.before
            logging.debug(f"Adding to output:[{out}]")
            self.expected_matched=expected[expected_index]
            self.output.append(self.process.before)
            
    def get_output(self, *args, **kwargs):
        """Run a command in bluetoothctl prompt, return output as a list of lines."""
        try:
            self.send(*args, **kwargs)
        except Exception as e:
            raise e   
        return self.process.before #self.read_last_output()

    def read_last_output(self):
        return self.output[-1]

    def read_full_output(self):
        return self.output


If we know the mac address of our device we can operate with it. For example we can check if it is in connected devices, in truste devices or in paired devices to remove it (removin diconnect, untrust and unpair device)

In [102]:
btctl=Bluetoothctl()
dev_mac="98:D3:31:00:0A:F6"
dev_connected=False
dev_trusted=False
dev_paired=False

res=btctl.get_output("devices Connected")
print(res)
if re.search(dev_mac,res):
    dev_connected=True

res=btctl.get_output("devices Trusted")
print(res)
if re.search(dev_mac,res):
    dev_trusted=True

res=btctl.get_output("devices Paired")
print(res)
if re.search(dev_mac,res):
    dev_paired=True
print(f"Device is connected={dev_connected},is trusted={dev_trusted}, is paired={dev_paired}")

2024-09-29 17:38:25,229 - DEBUG - Process created
2024-09-29 17:38:25,229 - DEBUG - Process created
2024-09-29 17:38:25,229 - DEBUG - Process created
2024-09-29 17:38:25,229 - DEBUG - Process created
2024-09-29 17:38:25,229 - DEBUG - Process created


[09/29/24 17:38:25] DEBUG    2024-09-29 17:38:25,229 - DEBUG - Process created                      ]8;id=913858;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=282959;file:///tmp/ipykernel_16595/1292906133.py#9\9]8;;\

[09/29/24 17:38:28] INFO     2024-09-29 17:38:28,387 - INFO - Sending command [devices Connected]  ]8;id=351957;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=452084;file:///tmp/ipykernel_16595/1292906133.py#22\22]8;;\
                             with expected=[['Invalid argument', 'Invalid command', '#', <class                    
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

                    DEBUG    2024-09-29 17:38:28,396 - DEBUG - expected_index=2                    ]8;id=265512;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=504485;file:///tmp/ipykernel_16595/1292906133.py#24\24]8;;\

                    DEBUG    2024-09-29 17:38:28,400 - DEBUG - Adding to output:[                  ]8;id=50386;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=584447;file:///tmp/ipykernel_16595/1292906133.py#30\30]8;;\
                             [[0;93mCHG[0m] Controller 4C:49:6C:0C:6B:4B Discovering: no                           
                             [?2004l                                                                               
                             Device 98:D3:31:00:0A:F6 Philips TAH4205                                              
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[Philips TAH4205][0m]                                                    

...

 
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[Philips TAH4205]


[09/29/24 17:38:34] INFO     2024-09-29 17:38:34,521 - INFO - Sending command [devices Paired]     ]8;id=882501;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=684918;file:///tmp/ipykernel_16595/1292906133.py#22\22]8;;\
                             with expected=[['Invalid argument', 'Invalid command', '#', <class                    
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

                    DEBUG    2024-09-29 17:38:34,528 - DEBUG - expected_index=2                    ]8;id=503792;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=655650;file:///tmp/ipykernel_16595/1292906133.py#24\24]8;;\

                    DEBUG    2024-09-29 17:38:34,532 - DEBUG - Adding to output:[ [?2004l          ]8;id=500656;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=931172;file:///tmp/ipykernel_16595/1292906133.py#30\30]8;;\
                             Device 98:D3:31:00:0A:F6 Philips TAH4205                                              
                             Device E5:1C:60:76:25:4B BT5.1 Mouse                                                  
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[Philips TAH4205][0m]                                                    

 
Device 98:D3:31:00:0A:F6 Philips TAH4205
Device E5:1C:60:76:25:4B BT5.1 Mouse
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[Philips TAH4205]
Device is connected=True,is trusted=False, is paired=True


Let's try to remove it if it is known by bluetoothctl

In [93]:
res=btctl.get_output("devices Paired")
print(res)
found=re.search(dev_mac,res)
if(dev_connected or dev_trusted or dev_paired):
    print("Device Found")
    # Let's removing it
    res=btctl.get_output(f"remove {dev_mac}",["Device has been removed",f"Device {dev_mac} not available"])
    print(res)
else:
    print("Device not found")

[09/29/24 17:25:15] INFO     2024-09-29 17:25:15,655 - INFO - Sending command [remove               ]8;id=293825;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=468488;file:///tmp/ipykernel_16595/415227010.py#23\23]8;;\
                             98:D3:31:00:0A:F6] with expected=[['Device has been removed', 'Device                 
                             98:D3:31:00:0A:F6 not available', 'Invalid argument', 'Invalid                        
                             command', '#', <class 'pexpect.exceptions.EOF'>]] and pause=[3]                       

                    DEBUG    2024-09-29 17:25:15,662 - DEBUG - expected_index=1                     ]8;id=778403;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=878930;file:///tmp/ipykernel_16595/415227010.py#25\25]8;;\

                    DEBUG    2024-09-29 17:25:15,665 - DEBUG - Adding to output:[ [?2004l           ]8;id=763970;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=407559;file:///tmp/ipykernel_16595/415227010.py#31\31]8;;\
                             ]                                                                                     

[09/29/24 17:24:33] INFO     2024-09-29 17:24:33,016 - INFO - Sending command [devices Paired] with ]8;id=438725;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=895636;file:///tmp/ipykernel_16595/415227010.py#23\23]8;;\
                             expected=[['Invalid argument', 'Invalid command', '#', <class                         
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

                    DEBUG    2024-09-29 17:24:33,022 - DEBUG - expected_index=2                     ]8;id=940009;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=422292;file:///tmp/ipykernel_16595/415227010.py#25\25]8;;\

                    DEBUG    2024-09-29 17:24:33,024 - DEBUG - Adding to output:[ [?2004l           ]8;id=25231;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=537390;file:///tmp/ipykernel_16595/415227010.py#31\31]8;;\
                             Device E5:1C:60:76:25:4B BT5.1 Mouse                                                  
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
Device E5:1C:60:76:25:4B BT5.1 Mouse
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[ZX-K22 BT5.1]
Device not found


Ensuring scan is off

In [94]:
try:
    res=btctl.get_output("scan off",["Discovery stopped"])
    print(res)
except Exception as e:
    print("It did not scan")


[09/29/24 17:26:50] INFO     2024-09-29 17:26:50,920 - INFO - Sending command [scan off] with       ]8;id=597436;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=395762;file:///tmp/ipykernel_16595/415227010.py#23\23]8;;\
                             expected=[['Discovery stopped', 'Invalid argument', 'Invalid command',                
                             '#', <class 'pexpect.exceptions.EOF'>]] and pause=[3]                                 

                    DEBUG    2024-09-29 17:26:50,926 - DEBUG - expected_index=3                     ]8;id=677683;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=557264;file:///tmp/ipykernel_16595/415227010.py#25\25]8;;\

                    DEBUG    2024-09-29 17:26:50,930 - DEBUG - Adding to output:[                   ]8;id=207254;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=501405;file:///tmp/ipykernel_16595/415227010.py#31\31]8;;\
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       


[ZX-K22 BT5.1]


Now start scanning around searchin for some new devices.

In [95]:
res=btctl.get_output("scan on",[dev_mac,"Discovery started"])
# Even I don0t care about output now
print(res)

[09/29/24 17:28:40] INFO     2024-09-29 17:28:40,404 - INFO - Sending command [scan on] with        ]8;id=208677;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=296344;file:///tmp/ipykernel_16595/415227010.py#23\23]8;;\
                             expected=[['98:D3:31:00:0A:F6', 'Discovery started', 'Invalid                         
                             argument', 'Invalid command', '#', <class 'pexpect.exceptions.EOF'>]]                 
                             and pause=[3]                                                                         

                    DEBUG    2024-09-29 17:28:40,411 - DEBUG - expected_index=4                     ]8;id=42416;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=867106;file:///tmp/ipykernel_16595/415227010.py#25\25]8;;\

                    DEBUG    2024-09-29 17:28:40,415 - DEBUG - Adding to output:[ [?2004l           ]8;id=116372;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=66964;file:///tmp/ipykernel_16595/415227010.py#31\31]8;;\
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 
[ZX-K22 BT5.1]


Tring to re-pairing the device. REMEMBER, DEVICE MUST BE IN PAIRING MODE AND VISIBLE TO CONTROLLER.

In [97]:
res=btctl.get_output(f"pair {dev_mac}",["Pairing successful"])
print(res)

[09/29/24 17:29:44] INFO     2024-09-29 17:29:44,147 - INFO - Sending command [pair                 ]8;id=718973;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=848019;file:///tmp/ipykernel_16595/415227010.py#23\23]8;;\
                             98:D3:31:00:0A:F6] with expected=[['Invalid argument', 'Invalid                       
                             command', '#', <class 'pexpect.exceptions.EOF'>]] and pause=[3]                       

                    DEBUG    2024-09-29 17:29:44,153 - DEBUG - expected_index=2                     ]8;id=258274;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=443551;file:///tmp/ipykernel_16595/415227010.py#25\25]8;;\

                    DEBUG    2024-09-29 17:29:44,155 - DEBUG - Adding to output:[                   ]8;id=415240;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=511470;file:///tmp/ipykernel_16595/415227010.py#31\31]8;;\
                             SetDiscoveryFilter success                                                            
                             Discovery started                                                                     
                             [[0;93mCHG[0m] Controller 4C:49:6C:0C:6B:4B Discovering: yes                          
                             [[0;92mNEW[0m] Device 7C:0A:3F:7D:CB:CB [TV] Samsung AU8070 43 TV                     
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB Modalias:                                     
                             bluetooth:v04E8p8080d0001                                                             
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB UUIDs:                                        
                             0000110a-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB UUIDs:                                        
                             0000110b-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB UUIDs:                                        
                             0000110c-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB UUIDs:                                        
                             0000110e-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB UUIDs:                                        
                             00001112-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB UUIDs:                                        
                             0000111f-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB UUIDs:                                        
                             00001200-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0xff19                  
                             (65305)                                                                               
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:                       
                               00 75 00 09 01 00 00 00 06 01 00 00 00 00 00 00  .u..............                   
                               00 00 00 00 00 00 00 00                          ........                           
                             [[0;92mNEW[0m] Device 98:D3:31:00:0A:F6 Philips TAH4205                               
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 UUIDs:                                        
                             0000111e-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 UUIDs:                                        
                             00001203-0000-1000-8000-00805f9b34fb                                                  
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 UUIDs:                                        
                             0000110b-0000-1000-8000-00805f9b34fb                                                  
           

 SetDiscoveryFilter success
Discovery started
[CHG] Controller 4C:49:6C:0C:6B:4B Discovering: yes
[NEW] Device 7C:0A:3F:7D:CB:CB [TV] Samsung AU8070 43 TV
[CHG] Device 7C:0A:3F:7D:CB:CB Modalias: bluetooth:v04E8p8080d0001
[CHG] Device 7C:0A:3F:7D:CB:CB UUIDs: 0000110a-0000-1000-8000-00805f9b34fb
[CHG] Device 7C:0A:3F:7D:CB:CB UUIDs: 0000110b-0000-1000-8000-00805f9b34fb
[CHG] Device 7C:0A:3F:7D:CB:CB UUIDs: 0000110c-0000-1000-8000-00805f9b34fb
[CHG] Device 7C:0A:3F:7D:CB:CB UUIDs: 0000110e-0000-1000-8000-00805f9b34fb
[CHG] Device 7C:0A:3F:7D:CB:CB UUIDs: 00001112-0000-1000-8000-00805f9b34fb
[CHG] Device 7C:0A:3F:7D:CB:CB UUIDs: 0000111f-0000-1000-8000-00805f9b34fb
[CHG] Device 7C:0A:3F:7D:CB:CB UUIDs: 00001200-0000-1000-8000-00805f9b34fb
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0xff19 (65305)
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:
  00 75 00 09 01 00 00 00 06 01 00 00 00 00 00 00  .u..............
  00 00 00 00 00 00 00 00                          ........   

In [98]:
res=btctl.get_output(f"connect {dev_mac}",["Connection successful"],5)
print(res)

[09/29/24 17:30:16] INFO     2024-09-29 17:30:16,837 - INFO - Sending command [connect              ]8;id=360212;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=233703;file:///tmp/ipykernel_16595/415227010.py#23\23]8;;\
                             98:D3:31:00:0A:F6] with expected=[['Connection successful', '#',                      
                             'Invalid argument', 'Invalid command', <class                                         
                             'pexpect.exceptions.EOF'>]] and pause=[5]                                             

                    DEBUG    2024-09-29 17:30:16,842 - DEBUG - expected_index=1                     ]8;id=387351;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=268450;file:///tmp/ipykernel_16595/415227010.py#25\25]8;;\

                    DEBUG    2024-09-29 17:30:16,846 - DEBUG - Adding to output:[ [[0;93mCHG[0m]    ]8;id=343488;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=396328;file:///tmp/ipykernel_16595/415227010.py#31\31]8;;\
                             Device 98:D3:31:00:0A:F6 Connected: yes                                               
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 Bonded: yes                                   
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 ServicesResolved: yes                         
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 Paired: yes                                   
                             Pairing successful                                                                    
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 ServicesResolved: no                          
                             [[0;93mCHG[0m] Device 98:D3:31:00:0A:F6 Connected: no                                 
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB AddressType: public                           
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB RSSI: 0xffffffb3 (-77)                        
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0xff19                  
                             (65305)                                                                               
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:                       
                               00 75 00 09 01 00 00 00 06 01 00 00 00 00 00 00  .u..............                   
                               00 00 00 00 00 00 00 00                          ........                           
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0x0075                  
                             (117)                                                                                 
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:                       
                               42 04 01 20 67 21 0d 00 02 01 2b 01 01 00 01 00  B.. g!....+.....                   
                               00 00 00 00 00 00 00 08                          ........                           
                             [?2004l                                                                               
                             Attempting to pair with 98:D3:31:00:0A:F6                                             
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 [CHG] Device 98:D3:31:00:0A:F6 Connected: yes
[CHG] Device 98:D3:31:00:0A:F6 Bonded: yes
[CHG] Device 98:D3:31:00:0A:F6 ServicesResolved: yes
[CHG] Device 98:D3:31:00:0A:F6 Paired: yes
Pairing successful
[CHG] Device 98:D3:31:00:0A:F6 ServicesResolved: no
[CHG] Device 98:D3:31:00:0A:F6 Connected: no
[CHG] Device 7C:0A:3F:7D:CB:CB AddressType: public
[CHG] Device 7C:0A:3F:7D:CB:CB RSSI: 0xffffffb3 (-77)
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0xff19 (65305)
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:
  00 75 00 09 01 00 00 00 06 01 00 00 00 00 00 00  .u..............
  00 00 00 00 00 00 00 00                          ........        
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0x0075 (117)
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:
  42 04 01 20 67 21 0d 00 02 01 2b 01 01 00 01 00  B.. g!....+.....
  00 00 00 00 00 00 00 08                          ........        

Attempting to pair with 98:D3:31:00:0A:F6
[ZX-K22 BT5.1]


In [99]:
try:
    res=btctl.get_output("scan off",["Discovery stopped"])
    print(res)
except Exception as e:
    print("It did not scan")

[09/29/24 17:32:00] INFO     2024-09-29 17:32:00,993 - INFO - Sending command [scan off] with       ]8;id=674536;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=652830;file:///tmp/ipykernel_16595/415227010.py#23\23]8;;\
                             expected=[['Discovery stopped', 'Invalid argument', 'Invalid command',                
                             '#', <class 'pexpect.exceptions.EOF'>]] and pause=[3]                                 

[09/29/24 17:32:01] DEBUG    2024-09-29 17:32:01,001 - DEBUG - expected_index=3                     ]8;id=432275;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=691239;file:///tmp/ipykernel_16595/415227010.py#25\25]8;;\

                    DEBUG    2024-09-29 17:32:01,005 - DEBUG - Adding to output:[ Failed to pair:   ]8;id=73663;file:///tmp/ipykernel_16595/415227010.py\415227010.py]8;;\:]8;id=991451;file:///tmp/ipykernel_16595/415227010.py#31\31]8;;\
                             org.bluez.Error.AlreadyExists                                                         
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0xff19                  
                             (65305)                                                                               
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:                       
                               00 75 00 09 01 00 00 00 06 01 00 00 00 00 00 00  .u..............                   
                               00 00 00 00 00 00 00 00                          ........                           
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0x0075                  
                             (117)                                                                                 
                             [[0;93mCHG[0m] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:                       
                               42 04 01 01 67 7c 0a 3f 7d cb cb 7e 0a 3f 7d cb  B...g|.?}..~.?}.                   
                               ca 01 00 00 00 00 00 00                          ........                           
                             [?2004l                                                                               
                             Attempting to connect to 98:D3:31:00:0A:F6                                            
                             [?2004h[0;94m[ZX-K22 BT5.1][0m]                                                       

 Failed to pair: org.bluez.Error.AlreadyExists
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0xff19 (65305)
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:
  00 75 00 09 01 00 00 00 06 01 00 00 00 00 00 00  .u..............
  00 00 00 00 00 00 00 00                          ........        
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Key: 0x0075 (117)
[CHG] Device 7C:0A:3F:7D:CB:CB ManufacturerData.Value:
  42 04 01 01 67 7c 0a 3f 7d cb cb 7e 0a 3f 7d cb  B...g|.?}..~.?}.
  ca 01 00 00 00 00 00 00                          ........        

Attempting to connect to 98:D3:31:00:0A:F6
[ZX-K22 BT5.1]


Another round on devices to see the situation

In [105]:
dev_connected=False
dev_trusted=False
dev_paired=False

res=btctl.get_output("devices Connected")
print(res)
if re.search(dev_mac,res):
    dev_connected=True

res=btctl.get_output("devices Trusted")
print(res)
if re.search(dev_mac,res):
    dev_trusted=True

res=btctl.get_output("devices Paired")
print(res)
if re.search(dev_mac,res):
    dev_paired=True
print(f"Device is connected={dev_connected},is trusted={dev_trusted}, is paired={dev_paired}")

[09/29/24 17:52:25] INFO     2024-09-29 17:52:25,727 - INFO - Sending command [devices Connected]  ]8;id=345142;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=761251;file:///tmp/ipykernel_16595/1292906133.py#22\22]8;;\
                             with expected=[['Invalid argument', 'Invalid command', '#', <class                    
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

                    DEBUG    2024-09-29 17:52:25,735 - DEBUG - expected_index=2                    ]8;id=742827;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=544014;file:///tmp/ipykernel_16595/1292906133.py#24\24]8;;\

                    DEBUG    2024-09-29 17:52:25,738 - DEBUG - Adding to output:[ [?2004l          ]8;id=64188;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=686981;file:///tmp/ipykernel_16595/1292906133.py#30\30]8;;\
                             Device 98:D3:31:00:0A:F6 Philips TAH4205                                              
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[Philips TAH4205][0m]                                                    

 
Device 98:D3:31:00:0A:F6 Philips TAH4205
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[Philips TAH4205]


[09/29/24 17:52:28] INFO     2024-09-29 17:52:28,793 - INFO - Sending command [devices Trusted]    ]8;id=121023;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=701895;file:///tmp/ipykernel_16595/1292906133.py#22\22]8;;\
                             with expected=[['Invalid argument', 'Invalid command', '#', <class                    
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

...

 
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[Philips TAH4205]


[09/29/24 17:52:31] INFO     2024-09-29 17:52:31,852 - INFO - Sending command [devices Paired]     ]8;id=914673;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=266848;file:///tmp/ipykernel_16595/1292906133.py#22\22]8;;\
                             with expected=[['Invalid argument', 'Invalid command', '#', <class                    
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

                    DEBUG    2024-09-29 17:52:31,857 - DEBUG - expected_index=2                    ]8;id=733683;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=732472;file:///tmp/ipykernel_16595/1292906133.py#24\24]8;;\

                    DEBUG    2024-09-29 17:52:31,859 - DEBUG - Adding to output:[ [?2004l          ]8;id=877457;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=141081;file:///tmp/ipykernel_16595/1292906133.py#30\30]8;;\
                             Device 98:D3:31:00:0A:F6 Philips TAH4205                                              
                             Device E5:1C:60:76:25:4B BT5.1 Mouse                                                  
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[Philips TAH4205][0m]                                                    

 
Device 98:D3:31:00:0A:F6 Philips TAH4205
Device E5:1C:60:76:25:4B BT5.1 Mouse
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[Philips TAH4205]
Device is connected=True,is trusted=False, is paired=True


As expected device is paired, connected but not trusted. Do you want trust it?

In [119]:
res=btctl.get_output("trust %s" % dev_mac,["trust succeeded"])
print(res)

[09/29/24 18:09:55] INFO     2024-09-29 18:09:55,090 - INFO - Sending command [trust               ]8;id=877488;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=707752;file:///tmp/ipykernel_16595/1292906133.py#22\22]8;;\
                             98:D3:31:00:0A:F6] with expected=[['trust succeeded', 'Invalid                        
                             argument', 'Invalid command', '#', <class 'pexpect.exceptions.EOF'>]]                 
                             and pause=[3]                                                                         

                    DEBUG    2024-09-29 18:09:55,098 - DEBUG - expected_index=0                    ]8;id=704305;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=730599;file:///tmp/ipykernel_16595/1292906133.py#24\24]8;;\

                    DEBUG    2024-09-29 18:09:55,101 - DEBUG - Adding to output:[ Changing         ]8;id=401938;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=474533;file:///tmp/ipykernel_16595/1292906133.py#30\30]8;;\
                             98:D3:31:00:0A:F6 ]                                                                   

 Changing 98:D3:31:00:0A:F6 


In [123]:
btctl=Bluetoothctl()
res=btctl.get_output("devices Connected")
print(res)

2024-09-29 18:13:01,186 - DEBUG - Process created
2024-09-29 18:13:01,186 - DEBUG - Process created
2024-09-29 18:13:01,186 - DEBUG - Process created
2024-09-29 18:13:01,186 - DEBUG - Process created
2024-09-29 18:13:01,186 - DEBUG - Process created


[09/29/24 18:13:01] DEBUG    2024-09-29 18:13:01,186 - DEBUG - Process created                      ]8;id=667802;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=512386;file:///tmp/ipykernel_16595/1292906133.py#9\9]8;;\

[09/29/24 18:13:04] INFO     2024-09-29 18:13:04,244 - INFO - Sending command [devices Connected]  ]8;id=49615;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=811349;file:///tmp/ipykernel_16595/1292906133.py#22\22]8;;\
                             with expected=[['Invalid argument', 'Invalid command', '#', <class                    
                             'pexpect.exceptions.EOF'>]] and pause=[3]                                             

                    DEBUG    2024-09-29 18:13:04,252 - DEBUG - expected_index=2                    ]8;id=956500;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=506472;file:///tmp/ipykernel_16595/1292906133.py#24\24]8;;\

                    DEBUG    2024-09-29 18:13:04,257 - DEBUG - Adding to output:[                  ]8;id=938202;file:///tmp/ipykernel_16595/1292906133.py\1292906133.py]8;;\:]8;id=476691;file:///tmp/ipykernel_16595/1292906133.py#30\30]8;;\
                             [?2004l                                                                               
                             Device 98:D3:31:00:0A:F6 Philips TAH4205                                              
                             Device E4:D7:00:02:73:8C ZX-K22 BT5.1                                                 
                             [?2004h[0;94m[Philips TAH4205][0m]                                                    



Device 98:D3:31:00:0A:F6 Philips TAH4205
Device E4:D7:00:02:73:8C ZX-K22 BT5.1
[Philips TAH4205]


### Last considerations
We can consider to not give back output when expected string is about Command or argument error to avoid misunderstanding with output.

## My use case
Why I needed it. I use a PC with windows 11 certified installed on bitlocker HD, but I want also to use a linux on that HW. So I run it from USB drive. When I switch to the other OS (from Win to Linux and viceversa) the previous connected bluetooth devices are listed as paired but they cannot be connected so I need to remove then, scan and pir them. Further, these operations does not work using UI (this in linux). In widows I must do the same but it works using UI method. I will try to use my script also on windows on wls.